In [ ]:
# auto reload magic
%load_ext autoreload
%autoreload 2

In [ ]:
from transformers import AutoProcessor, AutoModel
from datasets import load_dataset
import soundfile as sf
# processor = AutoProcessor.from_pretrained("facebook/hubert-base-ls960")
# processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
# model = AutoModel.from_pretrained("facebook/hubert-base-ls960")

In [ ]:
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

In [ ]:
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
ds = ds.map(map_to_array)

# batch_inputs = processor(ds["speech"][0:3], return_tensors="pt", padding=True)
# hidden_states = model(**batch_inputs).last_hidden_state

In [ ]:
from modeling.hubert import SpeechEncoder

speech_encoder = SpeechEncoder("facebook/hubert-base-ls960", 1024)


In [ ]:
x = speech_encoder(ds["speech"][0:3])

In [ ]:
x[0].shape, x[1].shape

In [ ]:
x[1]

In [ ]:
from transformers import AutoTokenizer
import transformers 
import torch
model = "TinyLlama/TinyLlama-1.1B-Chat-v0.4"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

CHAT_EOS_TOKEN_ID = 32002

prompt = "How to get in a good university?"
formatted_prompt = (
    f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
)


sequences = pipeline(
    formatted_prompt,
    do_sample=True,
    top_k=50,
    top_p = 0.9,
    num_return_sequences=1,
    repetition_penalty=1.1,
    max_new_tokens=1024,
    eos_token_id=CHAT_EOS_TOKEN_ID,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")


In [ ]:
import torch
from transformers import Speech2Text2Processor, SpeechEncoderDecoderModel
from datasets import load_dataset
import soundfile as sf

model = SpeechEncoderDecoderModel.from_pretrained("facebook/s2t-wav2vec2-large-en-de")
processor = Speech2Text2Processor.from_pretrained("facebook/s2t-wav2vec2-large-en-de")


def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch


ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
ds = ds.map(map_to_array)

inputs = processor(ds["speech"][0], sampling_rate=16_000, return_tensors="pt")
generated_ids = model.generate(inputs=inputs["input_values"], attention_mask=inputs["attention_mask"])

transcription = processor.batch_decode(generated_ids)